In [1]:
import pandas as pd
import numpy as np
import math
from datetime import date
import time
import os
from os.path import exists
import glob #para eliminar archivos dentro de carpeta
from datetime import datetime
import xlrd
# Para scrap
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
# Graficos
import plotly.graph_objects as go

In [2]:
for filename in glob.glob("../data/cin*"):
    os.remove(filename)

In [3]:
#Scrap
link='https://www.indec.gob.ar/indec/web/Nivel4-Tema-3-35-45'
# Descargar los datos de la web

def every_downloads_chrome(driver):
    '''Para ver cuando terminan las descargas'''
    if not driver.current_url.startswith("chrome://downloads"):
        driver.get("chrome://downloads/")
    return driver.execute_script("""
        var items = document.querySelector('downloads-manager')
            .shadowRoot.getElementById('downloadsList').items;
        if (items.every(e => e.state === "COMPLETE"))
            return items.map(e => e.fileUrl || e.file_url);
        """)

path='../selenium/chromedriver.exe'
carpeta_descarga=os.getcwd().replace('src','data')
#Con getcwd() se encuentra el path absoluto

chrome_options = webdriver.ChromeOptions()
prefs = {'download.default_directory' : carpeta_descarga,
        "directory_upgrade": True}
chrome_options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(path,options=chrome_options)

driver.get(link)
driver.maximize_window()

delay = 3 # seconds
try:
    myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//*[@id="2"]/div[2]/div[1]/div[2]/div/div/a')))
except TimeoutException:
    print("Loading took too much time!")

bp=driver.find_element(By.XPATH, '//*[@id="2"]/div[2]/div[1]/div[2]/div/div/a')
bp.click()

paths = WebDriverWait(driver, 300, 1).until(every_downloads_chrome)

driver.quit()

C:\Users\mbasualdo\AppData\Local\Temp\ipykernel_19708\1906382420.py:24: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path,options=chrome_options)


In [4]:
df=pd.read_excel('../data/cin_II_2022.xls',sheet_name=1, header=4)
df = df[df.columns.drop(list(df.filter(regex='Unnamed')))]
df = df[df.columns.drop(list(df.filter(regex='Total')))].T.reset_index()[['index',2]]

df.columns = ['trimestre','Bienes_y_servicios'] #set the header row as the df header
df.trimestre=([1,2,3,4]*len(df.trimestre))[:len(df.trimestre)]
fechas=range(2006,2006+math.ceil(len(df.trimestre)/4))
df['anio']=(np.repeat(fechas,4))[:len(df.trimestre)]
df['timestre_anio']=df.trimestre.astype(str)+'-'+df.anio.astype(str)
df=df[['timestre_anio','anio','trimestre','Bienes_y_servicios']]
df

df_anual=df.groupby('anio',as_index=False).sum().drop('trimestre',axis=1)
df_anual

,anio,Bienes_y_servicios
0,2006,13266.642573
1,2007,12570.579976
2,2008,13340.936973
3,2009,16652.818510
4,2010,12343.520649
5,2011,9198.443050
6,2012,10943.688063
7,2013,-693.680874
8,2014,900.177659
9,2015,-6600.221086


In [15]:
def plot_saldo_trimestral():
    x = df["timestre_anio"]

    #Datos
    cotizaciones_plot = go.Figure()
    cotizaciones_plot.add_trace(go.Scatter(name = "Saldo bienes y servicios BP", x=x, y=df["Bienes_y_servicios"], mode = "lines"))

    #Propiedades
    cotizaciones_plot.update_xaxes(showgrid=True)
    cotizaciones_plot.update_yaxes(title_text="Saldo",)
    cotizaciones_plot.update_layout(separators=",.", font_family="Georgia",
    height=600, width=1000,
    template = 'none',
    title_text=f"Saldo de bienes y servicios de la Balanza de Pagos<br><sup>Frecuencia trimestral. En millones de USD",
    legend = dict(yanchor="top",y=1.05, xanchor="left",x=0.36,orientation='h'))

    note = 'Fuente: INDEC'
    cotizaciones_plot.add_annotation(showarrow=False, text=note,font=dict(size=12), xref='paper',x=0.1,yref='paper', y=-0.17, 
                            xanchor='right', yanchor='auto', xshift=0, yshift=0,)

    #Marca de agua
    cotizaciones_plot.add_annotation(showarrow=False, text='@MartinBasualdo0', font=dict(size=16), font_family="arial", opacity=0.4,
        xref='paper', yref='paper', x=0.5, y=0.5)
    return cotizaciones_plot

def plot_saldo_anual():
    x = df_anual["anio"]

    #Datos
    cotizaciones_plot = go.Figure()
    cotizaciones_plot.add_trace(go.Scatter(name = "Saldo bienes y servicios BP", x=x, y=df_anual["Bienes_y_servicios"], mode = "lines"))

    #Propiedades
    cotizaciones_plot.update_xaxes(showgrid=True)
    cotizaciones_plot.update_yaxes(title_text="Saldo",)
    cotizaciones_plot.update_layout(separators=",.", font_family="Georgia",
    height=600, width=1000,
    template = 'none',
    title_text=f"Saldo de bienes y servicios de la Balanza de Pagos<br><sup>Frecuencia anual. En millones de USD",
    legend = dict(yanchor="top",y=1.05, xanchor="left",x=0.36,orientation='h'))

    note = 'Fuente: INDEC'
    cotizaciones_plot.add_annotation(showarrow=False, text=note,font=dict(size=12), xref='paper',x=0.1,yref='paper', y=-0.1, 
                            xanchor='right', yanchor='auto', xshift=0, yshift=0,)

    #Marca de agua
    cotizaciones_plot.add_annotation(showarrow=False, text='@MartinBasualdo0', font=dict(size=16), font_family="arial", opacity=0.4,
        xref='paper', yref='paper', x=0.5, y=0.5)
    return cotizaciones_plot

plot_saldo_anual()